In [1]:
import json
import os
import sys
from copy import deepcopy
from pathlib import Path
import pandas as pd
# from utils import load_json
import typing as ty
from collections import OrderedDict

def load_json(path: ty.Union[Path, str]) -> ty.Any:
    return json.loads(Path(path).read_text())

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# lib.env.OUTPUT_DIR = lib.env.PROJECT_DIR / 'output'
# lib.env.DATA_DIR = lib.env.PROJECT_DIR / 'data'

In [2]:
ALL_DATASETS = ['1_ALOI', '12_fault', '17_InternetAds', '19_landsat', '20_letter', \
            '22_magic.gamma', '23_mammography', '29_Pima', '30_satellite', '36_speech', \
                '41_Waveform', '47_yeast']
NORMAL_STRATEGIES = ['RandomSampling', 'LeastConfidence', 'MarginSampling', 'EntropySampling', \
        'LeastConfidenceDropout', 'MarginSamplingDropout', 'EntropySamplingDropout', \
        'BALDDropout', 'VarRatio', 'MeanSTD', 'MetaSampling']
EMBEDDING_STRATEGIES = ['KMeansSampling', 'KMeansSamplingGPU', 'KCenterGreedy', 'KCenterGreedyPCA', \
            'BadgeSampling', 'AdversarialBIM', 'AdversarialDeepFool']
WAAL_STRATEGY = 'WAAL'
LPL_STRATEGY = 'LossPredictionLoss'

DETAILS = ['init_labels', 'budget', 'batch_size']
BUDGET_RATIO_RANGE = [0.05, 0.1, 0.25, 0.5, 0.75]

In [3]:
def sort(df, by):
    if isinstance(by, str):
        by = [by]
    return df.sort_values(['n_objects'] + by, ascending=[True] + ['score' not in x for x in by]).reset_index(drop=True)

def load_record(output):
    """
    Load record from stats.json with a single random seed.
    Output: a dictionary with records
    """
    output = Path(output)
    if not output.exists():
        return None
    path = output / 'stats.json'
    if not path.exists():
        print(f'WARNING! This path does not exist: {path}')
        return None
    stats = load_json(path)
    metrics = stats.get('metrics')
    if metrics is None:
        return None

    dataset = stats['dataset']
    budget = stats['budget_ratio']
    batch_size = stats['batch_size']
    init_labels = stats['init_labeled']
    strategy = stats['strategy']
    base_model = stats['model']
    process = OrderedDict(stats['process'])

    r = {
        'dataset': dataset,
        'strategy': strategy,
        'base_model': base_model,
        'init_labels': init_labels,
        'budget': budget,
        'batch_size': batch_size,
        'all_outliers': process.popitem(last=True)[1][1], #TODO: make it more robust
        'F_aucroc': metrics['F-aucroc'],
        'F_aucpr': metrics['F_aucpr'],
        'F_f1': metrics['F_f1'],
    }
    
    return r

def aggregate(df):
    aggrs = dict(
        #TODO: make table that supports ensemble statistics
        # base_model=('base_model', 'first'),
        # budget=('budget', 'first'),
        init_labels=('init_labels', 'first'),
        batch_size=('batch_size', 'first'),
        all_outliers=('all_outliers', 'first'),
        F_aucroc=('F_aucroc', 'mean'),
        F_aucpr=('F_aucpr', 'mean'),
        F_f1=('F_f1', 'first'),
        count=('F_aucroc', 'count'),
    )
    if 'train_score' in df.columns:
        aggrs.update(dict(
            train_score=('train_score', 'mean'),
            train_std=('train_score', 'std'),
        ))
    df = df.groupby(['dataset', 'strategy', 'base_model', 'budget']).agg(**aggrs)
    df['count'] = df['count'].astype(int)
    return df.reset_index().fillna(0.0)

def format_scores(df, precision):
    def f(record):
        for k, v in list(record.items()):
            if isinstance(v, float):
                record[k] = round(v, precision)
        return record
    return df.apply(f, axis=1)

def make_df(outputs_and_names):
    """
    Make dataframe from a list of outputs and names.
    Output: unaggregated dataframe
    """
    df = []
    for output, strategy_name in outputs_and_names:
        record = load_record(output)
        if not record:
            continue
        df.append(record)
    # df = sort(pd.DataFrame(df).fillna(0.0), 'val_score').reset_index(drop=True)
    df = pd.DataFrame(df).fillna(0.0).reset_index(drop=True)
    df.sort_values(['F_aucroc', 'F_aucpr'], ascending=[True, True], inplace=True)
    # df.sort_values('F_aucroc', ascending=True, inplace=True)
    return df

def build_report_all(outputs_and_names):
    df = make_df(outputs_and_names)
    df = aggregate(df)
    df = format_scores(df, 4)
    df = df.set_index(['dataset'] + ['strategy'] + ['base_model'] + DETAILS)
    return df[['all_outliers', 'F_aucroc', 'F_aucpr', 'F_f1', 'count']]

def build_report_single(dataset, base_model, strategy, budget_ratio):
    output = f'output/{dataset}/{strategy}/{base_model}/{budget_ratio}'
    output = Path(output)
    if not output.exists():
        return None
    path = output / 'stats.json'
    if not path.exists():
        print(f'WARNING! This path does not exist: {path}')
        return None
    stats = load_json(path)
    process = stats.get('process')
    if process is None:
        return None
    df = pd.DataFrame.from_dict(process, orient='index')
    df.columns = ['all_X', 'all_outliers', 'new_X','new_outliers' ,'aucroc','aucpr','f1']
    # df = aggregate(df)
    df = format_scores(df, 4)
    return df

#### Comparison of different strategies with the same budget ratio (Base model: Devnet)

In [4]:
budget_ratio = 0.75  # 0.05 0.1 0.25 0.5 0.75, specify budget ratio here
BASE_MODEL = 'DevNet'
outputs_and_names = []
for strategy, base_model, datasets in [
    (s, BASE_MODEL, ALL_DATASETS) for s in NORMAL_STRATEGIES + EMBEDDING_STRATEGIES
]:
    for dataset in datasets:
        outputs_and_names.append((f'output/{dataset}/{strategy}/{base_model}/{budget_ratio}', strategy))

for strategy, base_model, datasets in [
    (WAAL_STRATEGY, 'WAAL', ALL_DATASETS),
    (LPL_STRATEGY, 'LPL', ALL_DATASETS),
]:
    for dataset in datasets:
        outputs_and_names.append((f'output/{dataset}/{strategy}/{base_model}/{budget_ratio}', strategy))

build_report_all(outputs_and_names)

all_outliers  \
dataset        strategy               base_model init_labels budget batch_size                 
12_fault       AdversarialBIM         DevNet     136         0.75   102                  433   
               AdversarialDeepFool    DevNet     136         0.75   102                  429   
               BALDDropout            DevNet     136         0.75   102                  434   
               BadgeSampling          DevNet     136         0.75   102                  372   
               EntropySampling        DevNet     136         0.75   102                  429   
               EntropySamplingDropout DevNet     136         0.75   102                  429   
               KCenterGreedy          DevNet     136         0.75   102                  396   
               KCenterGreedyPCA       DevNet     136         0.75   102                  393   
               KMeansSampling         DevNet     136         0.75   102                  408   
               KMeansSamplingGPU      DevNet     136         0.75   102                  386   
               LeastConfidence        DevNet     136         0.75   102                  429   
               LeastConfidenceDropout DevNet     136         0.75   102                  429   
               LossPredictionLoss     LPL        136         0.75   102                  450   
               MarginSampling         DevNet     136         0.75   102                  429   
               MarginSamplingDropout  DevNet     136         0.75   102                  429   
               MeanSTD                DevNet     136         0.75   102                  430   
               MetaSampling           DevNet     136         0.75   102                  403   
               RandomSampling         DevNet     136         0.75   102                  393   
               VarRatio               DevNet     136         0.75   102                  429   
               WAAL                   WAAL       136         0.75   102                  444   
17_InternetAds AdversarialBIM         DevNet     138         0.75   103                  213   
               AdversarialDeepFool    DevNet     138         0.75   103                  216   
               BALDDropout            DevNet     138         0.75   103                  215   
               EntropySampling        DevNet     138         0.75   103                  213   
               EntropySamplingDropout DevNet     138         0.75   103                  213   
               KCenterGreedy          DevNet     138         0.75   103                  239   
               KCenterGreedyPCA       DevNet     138         0.75   103                  244   
               KMeansSampling         DevNet     138         0.75   103                  244   
               KMeansSamplingGPU      DevNet     138         0.75   103                  218   
               LeastConfidence        DevNet     138         0.75   103                  213   
               LeastConfidenceDropout DevNet     138         0.75   103                  213   
               LossPredictionLoss     LPL        138         0.75   102                  252   
               MarginSampling         DevNet     138         0.75   103                  213   
               MarginSamplingDropout  DevNet     138         0.75   103                  213   
               MeanSTD                DevNet     138         0.75   103                  216   
               MetaSampling           DevNet     138         0.75   103                  234   
               RandomSampling         DevNet     138         0.75   103                  217   
               VarRatio               DevNet     138         0.75   103                  213   
               WAAL                   WAAL       138         0.75   103                  241   
19_landsat     AdversarialBIM         DevNet     451         0.75   338                  781   
               AdversarialDeepFool    DevNet    

#### Comparison of different strategies with the same budget ratio (Base model: XGBOD)

In [5]:
budget_ratio = 0.75  # 0.05 0.1 0.25 0.5 0.75, specify budget ratio here
BASE_MODEL = 'XGBOD'
outputs_and_names = []
for strategy, base_model, datasets in [
    (s, BASE_MODEL, ALL_DATASETS) for s in NORMAL_STRATEGIES
]:
    for dataset in datasets:
        outputs_and_names.append((f'output/{dataset}/{strategy}/{base_model}/{budget_ratio}', strategy))

for strategy, base_model, datasets in [
    (WAAL_STRATEGY, 'WAAL', ALL_DATASETS),
    (LPL_STRATEGY, 'LPL', ALL_DATASETS),
]:
    for dataset in datasets:
        outputs_and_names.append((f'output/{dataset}/{strategy}/{base_model}/{budget_ratio}', strategy))
build_report_all(outputs_and_names)

all_outliers  \
dataset        strategy               base_model init_labels budget batch_size                 
12_fault       BALDDropout            XGBOD      136         0.75   102                  397   
               EntropySampling        XGBOD      136         0.75   102                  461   
               EntropySamplingDropout XGBOD      136         0.75   102                  461   
               LeastConfidence        XGBOD      136         0.75   102                  461   
               LeastConfidenceDropout XGBOD      136         0.75   102                  461   
               LossPredictionLoss     LPL        136         0.75   102                  450   
               MarginSampling         XGBOD      136         0.75   102                  461   
               MarginSamplingDropout  XGBOD      136         0.75   102                  461   
               MeanSTD                XGBOD      136         0.75   102                  429   
               MetaSampling           XGBOD      136         0.75   102                  336   
               RandomSampling         XGBOD      136         0.75   102                  401   
               VarRatio               XGBOD      136         0.75   102                  461   
               WAAL                   WAAL       136         0.75   102                  444   
17_InternetAds BALDDropout            XGBOD      138         0.75   103                  217   
               EntropySampling        XGBOD      138         0.75   103                  257   
               EntropySamplingDropout XGBOD      138         0.75   103                  257   
               LeastConfidence        XGBOD      138         0.75   103                  257   
               LeastConfidenceDropout XGBOD      138         0.75   103                  257   
               LossPredictionLoss     LPL        138         0.75   102                  252   
               MarginSampling         XGBOD      138         0.75   103                  257   
               MarginSamplingDropout  XGBOD      138         0.75   103                  257   
               MeanSTD                XGBOD      138         0.75   103                  236   
               MetaSampling           XGBOD      138         0.75   103                  218   
               RandomSampling         XGBOD      138         0.75   103                  216   
               VarRatio               XGBOD      138         0.75   103                  257   
               WAAL                   WAAL       138         0.75   103                  241   
19_landsat     BALDDropout            XGBOD      451         0.75   338                  774   
               EntropySampling        XGBOD      451         0.75   338                  899   
               EntropySamplingDropout XGBOD      451         0.75   338                  899   
               LeastConfidence        XGBOD      451         0.75   338                  899   
               LeastConfidenceDropout XGBOD      451         0.75   338                  899   
               LossPredictionLoss     LPL        450         0.75   338                  922   
               MarginSampling         XGBOD      451         0.75   338                  899   
               MarginSamplingDropout  XGBOD      451         0.75   338                  899   
               MeanSTD                XGBOD      451         0.75   338                  885   
               MetaSampling           XGBOD      451         0.75   338                  567   
               RandomSampling         XGBOD      451         0.75   338                  786   
               VarRatio               XGBOD      451         0.75   338                  899   
               WAAL                   WAAL       451         0.75   338                  933   
1_ALOI         BALDDropout            XGBOD      700         0.75   525                  178   
               EntropySampling        XGBOD     

#### Comparison of different base models (DevNet vs. XGBOD)

In [16]:
budget_ratio = 0.05  # 0.05 0.1 0.25 0.5 0.75, specify budget ratio here
BASE_MODEL_1 = 'DevNet'
BASE_MODEL_2 = 'XGBOD'
outputs_and_names = []
for strategy, base_model, datasets in [
    (s, BASE_MODEL_1, ALL_DATASETS) for s in NORMAL_STRATEGIES
]:
    for dataset in datasets:
        outputs_and_names.append((f'output/{dataset}/{strategy}/{base_model}/{budget_ratio}', strategy))

for strategy, base_model, datasets in [
    (s, BASE_MODEL_2, ALL_DATASETS) for s in NORMAL_STRATEGIES
]:
    for dataset in datasets:
        outputs_and_names.append((f'output/{dataset}/{strategy}/{base_model}/{budget_ratio}', strategy))

build_report_all(outputs_and_names)

all_outliers  \
dataset        strategy               base_model init_labels budget batch_size                 
12_fault       BALDDropout            DevNet     136         0.05   32                    71   
                                      XGBOD      136         0.05   32                    55   
               EntropySampling        DevNet     136         0.05   32                    85   
                                      XGBOD      136         0.05   32                    78   
               EntropySamplingDropout DevNet     136         0.05   32                    85   
                                      XGBOD      136         0.05   32                    78   
               LeastConfidence        DevNet     136         0.05   32                    85   
                                      XGBOD      136         0.05   32                    78   
               LeastConfidenceDropout DevNet     136         0.05   32                    85   
                                      XGBOD      136         0.05   32                    78   
               MarginSampling         DevNet     136         0.05   32                    85   
                                      XGBOD      136         0.05   32                    78   
               MarginSamplingDropout  DevNet     136         0.05   32                    85   
                                      XGBOD      136         0.05   32                    78   
               MeanSTD                DevNet     136         0.05   32                    63   
                                      XGBOD      136         0.05   32                    82   
               MetaSampling           DevNet     136         0.05   32                   146   
                                      XGBOD      136         0.05   32                   107   
               RandomSampling         DevNet     136         0.05   32                    59   
                                      XGBOD      136         0.05   32                    64   
               VarRatio               DevNet     136         0.05   32                    85   
                                      XGBOD      136         0.05   32                    78   
1_ALOI         BALDDropout            DevNet     700         0.05   35                    24   
                                      XGBOD      700         0.05   35                    29   
               EntropySampling        DevNet     700         0.05   35                    36   
                                      XGBOD      700         0.05   35                    34   
               EntropySamplingDropout DevNet     700         0.05   35                    36   
                                      XGBOD      700         0.05   35                    29   
               LeastConfidence        DevNet     700         0.05   35                    36   
                                      XGBOD      700         0.05   35                    30   
               LeastConfidenceDropout DevNet     700         0.05   35                    36   
                                      XGBOD      700         0.05   35                    30   
               MarginSampling         DevNet     700         0.05   35                    36   
                                      XGBOD      700         0.05   35                    30   
               MarginSamplingDropout  DevNet     700         0.05   35                    36   
                                      XGBOD      700         0.05   35                    30   
               MeanSTD                DevNet     700         0.05   35                    28   
                                      XGBOD      700         0.05   35                    26   
               MetaSampling           DevNet     700         0.05   35                    37   
                                      XGBOD      700         0.05   35                    43   
               RandomSampling         DevNet    

#### Comparison of different budgets

In [19]:
# specify your input here
DATASET = '17_InternetAds' # 17_InternetAds, 20_letter, 12_fault, 22_magic.gamma
BASE_MODEL = 'LPL' # LPL, WAAL, XGBOD, DevNet
STRATEGY = 'LossPredictionLoss'  # MetaSampling, BALDDropout, BadgeSampling, RandomSampling, WAAL, LossPredictionLoss, BALD
outputs_and_names = []

for ratio in BUDGET_RATIO_RANGE:
    outputs_and_names.append((f'output/{DATASET}/{STRATEGY}/{BASE_MODEL}/{ratio}', STRATEGY))

build_report_all(outputs_and_names)

all_outliers  \
dataset        strategy           base_model init_labels budget batch_size                 
17_InternetAds LossPredictionLoss LPL        138         0.10   32                    87   
                                                         0.25   34                   177   
                                                         0.50   68                   232   
                                                         0.75   102                  252   

                                                                            F_aucroc  \
dataset        strategy           base_model init_labels budget batch_size             
17_InternetAds LossPredictionLoss LPL        138         0.10   32            0.9406   
                                                         0.25   34            0.9560   
                                                         0.50   68            0.9614   
                                                         0.75   102           0.9677   

                                                                            F_aucpr  \
dataset        strategy           base_model init_labels budget batch_size            
17_InternetAds LossPredictionLoss LPL        138         0.10   32           0.8828   
                                                         0.25   34           0.9194   
                                                         0.50   68           0.9182   
                                                         0.75   102          0.9170   

                                                                                       F_f1  \
dataset        strategy           base_model init_labels budget batch_size                    
17_InternetAds LossPredictionLoss LPL        138         0.10   32          [0.968 ,0.8531]   
                                                         0.25   34          [0.9741,0.8826]   
                                                         0.50   68          [0.9676,0.861 ]   
                                                         0.75   102         [0.9603,0.8304]   

                                                                            count  
dataset        strategy           base_model init_labels budget batch_size         
17_InternetAds LossPredictionLoss LPL        138         0.10   32              1  
                                                         0.25   34              1  
                                                         0.50   68              1  
                                                         0.75   102             1

#### Single active learning process

In [80]:
budget_ratio = 0.5 # 0.05 0.1 0.25 0.5 0.75, specify budget ratio here
DATASET = '12_fault'
BASE_MODEL = 'DevNet'
STRATEGY = 'BALDDropout'

build_report_single(DATASET, BASE_MODEL, STRATEGY, budget_ratio)

,all_X,all_outliers,new_X,new_outliers,aucroc,aucpr,f1
round_0,1358,38,68,38,0.6538,0.5086,"[0.7923,0.1638]"
round_1,1358,68,68,30,0.6711,0.5125,"[0.7923,0.1638]"
round_2,1358,95,68,27,0.6970,0.5240,"[0.7923,0.1638]"
round_3,1358,130,68,35,0.7078,0.5600,"[0.7944,0.1724]"
round_4,1358,164,68,34,0.7162,0.5590,"[0.7966,0.181 ]"
round_5,1358,202,68,38,0.7177,0.5661,"[0.7966,0.181 ]"
round_6,1358,248,68,46,0.7218,0.5677,"[0.7966,0.181 ]"
round_7,1358,288,68,40,0.7305,0.5831,"[0.7944,0.1724]"
round_8,1358,306,68,18,0.7266,0.5809,"[0.7966,0.181 ]"
round_9,1358,343,68,37,0.7216,0.5824,"[0.803 ,0.2069]"
